# Cryptocurrency trading bot.

In [ ]:
##%pip install python-binance==0.7.5
#%pip install python-binance

In [ ]:
!python bootstrap.py

In [1]:
from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets
from cryptocurrency.ohlcvs import download_pairs
from cryptocurrency.resample import resample
from cryptocurrency.volume_conversion import add_rolling_volumes
from tqdm import tqdm
import pandas as pd

def bootstrap_loggers(client, assets, pairs=None, additional_intervals=None, upsampled_intervals=None, 
                      download_interval='1m', exchange_info=None, as_pair=False):
    log_file = 'crypto_logs/crypto_output_log_{}.txt'
    period = 2880 if download_interval == '1m' else 1000
    second_period = 60 if download_interval == '1m' else None
    base_interval = download_interval + 'in' if download_interval[-1] == 'm' else download_interval
    frequency_1d = pd.tseries.frequencies.to_offset('1d')
    frequency = pd.tseries.frequencies.to_offset(base_interval)
    pairs[base_interval] = download_pairs(client=client, assets=assets, interval=download_interval, 
                                          period=period, second_period=second_period)
    if not as_pair:
        pairs[base_interval] = convert_ohlcvs_from_pairs_to_assets(pairs[base_interval], exchange_info)
    if frequency < frequency_1d:
        pairs[base_interval] = add_rolling_volumes(pairs[base_interval])
    if download_interval == '1m':
        pairs[base_interval] = pairs[base_interval].loc[pairs[base_interval].dropna().first_valid_index():]
    pairs[base_interval].to_csv(log_file.format(base_interval))
    if additional_intervals is not None:
        for additional_interval in tqdm(additional_intervals, unit=' pair'):
            pairs[additional_interval] = resample(pairs[base_interval], interval=additional_interval)
            pairs[additional_interval].to_csv(log_file.format(additional_interval))
    if upsampled_intervals is not None:
        for subminute_interval in tqdm(upsampled_intervals, unit=' pair'):
            pairs[subminute_interval] = pairs[base_interval].tail(25)
            pairs[subminute_interval] = pairs[subminute_interval].resample(subminute_interval).agg('max')
            pairs[subminute_interval] = pairs[subminute_interval].fillna(method='pad')
            pairs[subminute_interval].to_csv(log_file.format(subminute_interval))
    return pairs

In [2]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table, get_new_tickers
#from cryptocurrency.bootstrap import bootstrap_loggers
import os
import shutil

as_pair = False
directory = 'crypto_logs'
if os.path.exists(directory):
    shutil.rmtree(directory)
os.mkdir(directory)
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory=directory)
exchange_info = exchange.info
offset_s = get_timezone_offset_in_seconds()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, as_pair=True)
assets = get_new_tickers(conversion_table=conversion_table)
pairs = bootstrap_loggers(client=client, assets=assets, pairs={}, 
                          download_interval='1h', exchange_info=exchange_info, as_pair=as_pair)
df = pairs['1h']
df

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1428/1428 [14:02<00:00,  1.70 pair/s]
1428 named pair [01:15, 18.83 named pair/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1428/1428 [00:05<00:00, 277.61 pair conversion/s]
1428 named pair [00:30, 46.66 named pair/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1428/1428 [00:05<00:00, 248.76 pair conversion/s]
1428 named pair [00:29, 47.88 named pair/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1428/1428 [00:00<00:00, 4923.15 pair/s]
1428 named pair [00:43, 33.18 named pair/s]
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1428/1428 [00:00<00

symbol,1INCH,AAVE,ACA,ACH,ACM,ADA,ADX,AERGO,AGIX,AGLD,...,XTZ,XVG,XVS,YFI,YFII,YGG,ZEC,ZEN,ZIL,ZRX
pair,open,open,open,open,open,open,open,open,open,open,...,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume,rolling_quote_volume
date,,,,,,,,,,,,,,,,,,,,,
2022-09-26 14:00:00,0.631,76.0,0.1998,0.01178,3.866,0.4471,0.1692,3.423578e-10,1.391317e-10,0.324,...,1.044655e+05,9.370694e+03,3.024876e+04,3.162398e+05,106303.299609,8.452316e+04,1.039808e+06,6.938317e+04,3.205858e+05,2.052706e+04
2022-09-26 15:00:00,0.629,76.1,0.1987,0.01177,3.861,0.4457,0.1697,3.432946e-10,1.410800e-10,0.321,...,1.741897e+05,2.188356e+04,5.755895e+04,4.551650e+05,143849.855095,1.965491e+05,1.459670e+06,1.197758e+05,5.149670e+05,4.622175e+04
2022-09-26 16:00:00,0.631,75.9,0.1987,0.01169,3.854,0.4456,0.1709,3.432133e-10,1.417848e-10,0.321,...,3.549661e+05,4.161121e+04,7.198639e+04,6.310227e+05,190312.852397,2.701413e+05,1.752447e+06,1.903833e+05,7.351587e+05,7.516724e+04
2022-09-26 17:00:00,0.626,75.3,0.1977,0.01165,3.832,0.4446,0.1706,3.432002e-10,1.398999e-10,0.322,...,4.461144e+05,5.104893e+04,8.435591e+04,6.817476e+05,218509.991748,3.506587e+05,1.912403e+06,2.248743e+05,1.605094e+06,1.275128e+05
2022-09-26 18:00:00,0.625,75.5,0.1980,0.01166,3.841,0.4456,0.1766,3.426770e-10,1.391635e-10,0.322,...,5.575293e+05,5.533403e+04,9.962439e+04,8.424424e+05,254073.638119,4.284485e+05,2.065334e+06,2.499001e+05,1.728701e+06,1.534719e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-07 01:00:00,0.630,85.9,0.1674,0.01193,3.310,0.4029,0.1763,3.301576e-10,1.502456e-10,0.350,...,5.537100e+06,1.429040e+06,1.174067e+06,3.362739e+06,767094.193261,9.755920e+06,5.151356e+06,1.796842e+06,1.759626e+07,1.789738e+06
2022-11-07 02:00:00,0.632,86.7,0.1667,0.01189,3.338,0.4091,0.1769,3.268784e-10,1.472382e-10,0.357,...,5.516941e+06,1.429712e+06,1.174440e+06,3.496102e+06,739069.500965,8.944645e+06,5.028259e+06,1.766960e+06,1.776200e+07,1.719259e+06


In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table, get_new_tickers
#from cryptocurrency.bootstrap import bootstrap_loggers
import os
import shutil

from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets
#from cryptocurrency.ohlcvs import download_pairs
from cryptocurrency.resample import resample
from cryptocurrency.volume_conversion import add_rolling_volumes
from tqdm import tqdm
import pandas as pd

from cryptocurrency.ohlcv import download_pair
import time

as_pair = False
directory = 'crypto_logs'
if os.path.exists(directory):
    shutil.rmtree(directory)
os.mkdir(directory)
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory=directory)
exchange_info = exchange.info
offset_s = get_timezone_offset_in_seconds()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, as_pair=True)
assets = get_new_tickers(conversion_table=conversion_table)
assets = assets[-50:]
#pairs = bootstrap_loggers(client=client, assets=assets, pairs={}, 
#                          download_interval='1h', exchange_info=exchange_info, as_pair=True)

pairs = {}
additional_intervals = None
upsampled_intervals = None
download_interval = '1h'
exchange_info = None
as_pair = True
log_file = 'crypto_logs/crypto_output_log_{}.txt'
period = 2880 if download_interval == '1m' else 1000
second_period = 60 if download_interval == '1m' else None
base_interval = download_interval + 'in' if download_interval[-1] == 'm' else download_interval
frequency_1d = pd.tseries.frequencies.to_offset('1d')
frequency = pd.tseries.frequencies.to_offset(base_interval)
#pairs[base_interval] = download_pairs(client=client, assets=assets, interval=download_interval, 
#                                      period=period, second_period=second_period)

def get_timezone_offset_in_seconds():
    is_dst = time.localtime().tm_isdst
    timezone = time.tzname[is_dst]
    offset_s = time.altzone if is_dst else time.timezone
    #offset = (offset_s / 60 / 60)
    return offset_s

def named_pairs_to_df(assets, pairs):
    df = pd.DataFrame()
    column_names = pairs[0].columns.tolist()
    for (asset, pair) in tqdm(zip(assets, pairs), unit=' named pair'):
        columns = [(asset, column) for column in column_names]
        pair.columns = pd.MultiIndex.from_tuples(columns, names=['symbol', 'pair'])
        df = pd.concat([df, pair], axis='columns')
    return df

interval = '1h'
period = 1000
second_period = None

offset_s = get_timezone_offset_in_seconds()
#pairs_at_interval = download_pairs_helper(period=period, offset_s=offset_s)

period = 1000
#offset_s = 0
pairs_at_interval = [download_pair(client=client, symbol=symbol, interval=interval, 
                                   period=period, offset_s=offset_s) 
                     for symbol in tqdm(assets, unit=' pair')]
pairs_at_interval = named_pairs_to_df(assets, pairs_at_interval)
pairs_at_interval = pairs_at_interval.sort_index(axis='index')
pairs_at_interval.columns = pairs_at_interval.columns.swaplevel(0, 1)
pairs_at_interval = pairs_at_interval[['open', 'high', 'low', 'close', 'base_volume', 'quote_volume']]
pairs_at_interval.columns = pairs_at_interval.columns.swaplevel(0, 1)
pairs_at_interval = pairs_at_interval.sort_index(axis='columns').sort_index(axis='index')

pairs_at_interval = pairs_at_interval[~pairs_at_interval.index.duplicated(keep='last')]
pairs_at_interval.iloc[:,pairs_at_interval.columns.get_level_values(1) == 'base_volume'] = \
    pairs_at_interval.xs('base_volume', axis=1, level=1).fillna(0)
pairs_at_interval.iloc[:,pairs_at_interval.columns.get_level_values(1) == 'quote_volume'] = \
    pairs_at_interval.xs('quote_volume', axis=1, level=1).fillna(0)
pairs_at_interval = pairs_at_interval.fillna(method='pad')

#pairs_at_interval = pairs.copy()
pairs = {}
#pairs[base_interval] = download_pairs(client=client, assets=assets, interval=download_interval, 
#                                      period=period, second_period=second_period)
pairs[base_interval] = pairs_at_interval

#if not as_pair:
#    pairs[base_interval] = convert_ohlcvs_from_pairs_to_assets(pairs[base_interval], exchange_info)
#if frequency < frequency_1d:
#    pairs[base_interval] = add_rolling_volumes(pairs[base_interval])
#if download_interval == '1m':
#    pairs[base_interval] = pairs[base_interval].loc[pairs[base_interval].dropna().first_valid_index():]
#pairs[base_interval].to_csv(log_file.format(base_interval))
#if additional_intervals is not None:
#    for additional_interval in tqdm(additional_intervals, unit=' pair'):
#        pairs[additional_interval] = resample(pairs[base_interval], interval=additional_interval)
#        pairs[additional_interval].to_csv(log_file.format(additional_interval))
#if upsampled_intervals is not None:
#    for subminute_interval in tqdm(upsampled_intervals, unit=' pair'):
#        pairs[subminute_interval] = pairs[base_interval].tail(25)
#        pairs[subminute_interval] = pairs[subminute_interval].resample(subminute_interval).agg('max')
#        pairs[subminute_interval] = pairs[subminute_interval].fillna(method='pad')
#        pairs[subminute_interval].to_csv(log_file.format(subminute_interval))

df = pairs['1h']
df

In [ ]:
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_timezone_offset_in_seconds
from cryptocurrency.conversion_table import get_conversion_table, get_new_tickers
#from cryptocurrency.bootstrap import bootstrap_loggers
import os
import shutil

from cryptocurrency.conversion import convert_ohlcvs_from_pairs_to_assets
#from cryptocurrency.ohlcvs import download_pairs
from cryptocurrency.resample import resample
from cryptocurrency.volume_conversion import add_rolling_volumes
from tqdm import tqdm
import pandas as pd

from cryptocurrency.ohlcv import download_pair
import time

as_pair = False
directory = 'crypto_logs'
if os.path.exists(directory):
    shutil.rmtree(directory)
os.mkdir(directory)
authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
client = authenticator.spot_client
exchange = Cryptocurrency_exchange(client=client, directory=directory)
exchange_info = exchange.info
offset_s = get_timezone_offset_in_seconds()
conversion_table = get_conversion_table(client=client, exchange_info=exchange_info, 
                                        offset_s=offset_s, as_pair=True)
assets = get_new_tickers(conversion_table=conversion_table)
assets = assets[-50:]
#pairs = bootstrap_loggers(client=client, assets=assets, pairs={}, 
#                          download_interval='1h', exchange_info=exchange_info, as_pair=True)

pairs = {}
additional_intervals = None
upsampled_intervals = None
download_interval = '1h'
exchange_info = None
as_pair = True
log_file = 'crypto_logs/crypto_output_log_{}.txt'
period = 2880 if download_interval == '1m' else 1000
second_period = 60 if download_interval == '1m' else None
base_interval = download_interval + 'in' if download_interval[-1] == 'm' else download_interval
frequency_1d = pd.tseries.frequencies.to_offset('1d')
frequency = pd.tseries.frequencies.to_offset(base_interval)
#pairs[base_interval] = download_pairs(client=client, assets=assets, interval=download_interval, 
#                                      period=period, second_period=second_period)

def get_timezone_offset_in_seconds():
    is_dst = time.localtime().tm_isdst
    timezone = time.tzname[is_dst]
    offset_s = time.altzone if is_dst else time.timezone
    #offset = (offset_s / 60 / 60)
    return offset_s

def named_pairs_to_df(assets, pairs):
    df = pd.DataFrame()
    column_names = pairs[0].columns.tolist()
    for (asset, pair) in tqdm(zip(assets, pairs), unit=' named pair'):
        columns = [(asset, column) for column in column_names]
        pair.columns = pd.MultiIndex.from_tuples(columns, names=['symbol', 'pair'])
        df = pd.concat([df, pair], axis='columns')
    return df

interval = '1h'
period = 1000
second_period = None

offset_s = get_timezone_offset_in_seconds()
#pairs_at_interval = download_pairs_helper(period=period, offset_s=offset_s)

period = 1000
#offset_s = 0
pairs_at_interval = [download_pair(client=client, symbol=symbol, interval=interval, 
                                   period=period, offset_s=offset_s) 
                     for symbol in tqdm(assets, unit=' pair')]
pairs_at_interval

In [ ]:
df1 = pairs_at_interval[0]
df1

In [ ]:
x = [pair_at_interval.index.has_duplicates for (i, pair_at_interval) in enumerate(pairs_at_interval)]
x

In [ ]:
df1.index.has_duplicates

In [ ]:
df = named_pairs_to_df(assets, pairs_at_interval)
df

In [ ]:
df1.index

In [ ]:
timeseries_size = df.shape[0]
mid_series = timeseries_size // 2
frequency_1 = pd.tseries.frequencies.to_offset((df.index[-(mid_series)+1:] - df.index[-(mid_series):-1]).min())
frequency_2 = pd.tseries.frequencies.to_offset((df.index[1:mid_series] - df.index[:(mid_series)-1]).min())
frequency_1d = pd.tseries.frequencies.to_offset('1d')
frequency = frequency_2 if frequency_1 < frequency_2 else frequency_1
if frequency_1 != frequency_2:
    if (frequency_1d > frequency_1) and (frequency_1d > frequency_2):
        df.index = pd.date_range(end=df.index[-1], periods=timeseries_size, freq=frequency)
df

In [ ]:
df1.index = pd.date_range(end=df1.index[-1], periods=timeseries_size, freq=frequency)

In [ ]:
df1.index[-1]

In [ ]:
df1.index.freq = frequency
df1

In [ ]:
df1.index.has_duplicates

In [ ]:
df1.tz_localize(tz='UTC', ambiguous='raise').index.duplicated

In [ ]:
pairs_at_interval[~pairs_at_interval.index.duplicated(keep='last')]